<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/youtube_subtitle/whisper-caption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
Here we install the required packages for this application. Additionally, we will remove a single line from the ImageMagick policy that would have prevented this code from running, create our experiments directory, and restart the kernel.

> Note: We need to restart the kernel due to an odd behavior from MoviePy that stops this from working in the same session as the install.

> Just move onto the next section of this notebook after running the install cell below.

> We should be especially wary of this if we intend to 'Run All' cells, as it will catch here.

In [1]:

from google.colab import drive
import os
from pathlib import Path

global gdrive_fpath
drive_mounted = False
gdrive_fpath = '.'
local_path = '/content/'

mount_gdrive = True # @param{type:"boolean"}
if mount_gdrive : # and not drive_mounted:
    from google.colab import drive

    gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
    gdrive_subdirectory = 'MyDrive/YouTube_Auto_Subtitle' #@param{type:"string"}
    gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
    print ("gdrive path is :",gdrive_fpath)
   # Mount Google Drive
    if not os.path.isdir(gdrive_mountpoint):
     # If not, mount the drive
       drive.mount(gdrive_mountpoint)
       if not os.path.exists(gdrive_fpath):
          os.makedirs(gdrive_fpath)
          os.chdir(gdrive_fpath)
    else:
          print("Drive is already mounted.")
else:
   Folder_fpath ='/content/' #@param{type:"string"}
   #gdrive_subdirectory = 'MyDrive/ChatGPT_Paper_wrting' #@param{type:"string"}
   gdrive_fpath = Folder_fpath
   os.chdir(gdrive_fpath)
folder_path = gdrive_fpath

gdrive path is : /content/drive/MyDrive/YouTube_Auto_Subtitle
Mounted at /content/drive/


In [ ]:
%cd /content/
!git clone https://github.com/gradient-ai/Whisper-AutoCaption

%cd Whisper-AutoCaption
import os
!pip install -r requirements.txt
!pip install --upgrade git+https://github.com/openai/whisper.git

!pip install yt-dlp
!pip install moviepy --upgrade
!apt-get update
!apt install imagemagick -y
# remove line 88 of vim ~/../etc/ImageMagick-6/policy.xml to run MoviePy
!sed -i '88d' ~/../etc/ImageMagick-6/policy.xml
!mkdir experiments
#os._exit(00)

/content
Cloning into 'Whisper-AutoCaption'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 352 (delta 58), reused 89 (delta 26), pack-reused 223
Receiving objects: 100% (352/352), 3.20 MiB | 23.40 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/Whisper-AutoCaption
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-p_s6czmg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-p_s6czmg
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sh

# The function

The `subtitle_video` function does all the work for us to autocaption our supplied video with the generated text captions from Whisper at the correct time stamps.

This works for both youtube links and videos uploaded directly to this Notebook, and will automatically scale the size of the captions to the video size.

See the params notes section of the start of the function and the function call below for more details on the arguments for this function.

In [ ]:
## Imports
from __future__ import unicode_literals
from yt_dlp import YoutubeDL
import yt_dlp
from IPython.display import Video
import whisper
import cv2
import pandas as pd
from moviepy.editor import VideoFileClip
import moviepy.editor as mp
from IPython.display import display, Markdown
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip
import os

import cv2
import subprocess
import json

import yt_dlp
from whisper.utils import get_writer



def get_video_title(url):
    ydl_opts = {
        'skip_download': True, # We don't want to download the video
        'quiet': True, # We don't want to print everything to the console
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info.get('title', None)

def subtitle_video(download, url, aud_opts, vid_opts, model_type, name, audio_file, input_file, output, lang, uploaded_vid = None):
# ------------------------------------------------------------------------------------------------------------------------------
#     Params:
# ------------------------------------------------------------------------------------------------------------------------------
#     download:      bool, this tells your function if you are downloading a youtube video
#     url: str,      str, the URL of youtube video to download if download is True
#     aud_opts:      dict, audio file youtube-dl options
#     vid_opts:      dict, video file youtube-dl options
#     model_type:    str, which pretrained model to download. Options are:
#                    ['tiny', 'small', 'base', 'medium','large','tiny.en', 'small.en', 'base.en', 'medium.en']
#                    More details about model_types can be found in table in original repo here:
#                    https://github.com/openai/whisper#Available-models-and-languages
#.    name:          str, name of directory to store files in in experiments folder
#     audio_file:    str, path to extracted audio file for Whisper
#     input_file:    str, path to video file for MoviePy to caption
#     output:        str, destination of final output video file
#     uploaded_vid:  str, path to uploaded video file if download is False
#
#--------------------------------------------------------------------------------------------------------------------------------
#     Returns:       An annotated video with translated captions into english, saved to name/output
#--------------------------------------------------------------------------------------------------------------------------------

    ## First, this checks if your expermiment name is taken. If not, it will create the directory.
    ## Otherwise, we will be prompted to retry with a new name
    try:
        os.mkdir(f'{name}')
        print('Starting AutoCaptioning...')
        print(f'Results will be stored in :{name}')

    except:
       pass #print('Choose another folder name! This one already has files in it.')

    ## Use audio and video options for youtube-dl if downloading from youtube
    vid_opts['outtmpl'] = f'{name}/{input_file}'
    aud_opts['outtmpl'] = f'{name}/{audio_file}'

    URLS = [url]
    if download:
        with YoutubeDL(aud_opts) as ydl:
            ydl.download(url)
            title = get_video_title(url)
        with YoutubeDL(vid_opts) as ydl:
            ydl.download(URLS)

    else:
        # Use local clip if not downloading from youtube
        my_clip = mp.VideoFileClip(uploaded_vid)
        my_clip.write_videofile(f'{name}/{input_file}')
        my_clip.audio.write_audiofile(f'{name}/{audio_file}')



    title  = "".join(c if c.isalnum() else "_" for c in title).rstrip("_")
    output = f'{name}/{title}.mp4'
    # Instantiate whisper model using model_type variable
    model = whisper.load_model(model_type)

    # Get text from speech for subtitles from audio file
    result = model.transcribe(f'''{name}/{audio_file}''', task = 'translate', language = lang)
    srt_writer = get_writer("srt", f"{name}/{audio_file}")
    srt_writer(result, f'{name}/{title}.str')

    # create Subtitle dataframe, and save it
    dict1 = {'start':[], 'end':[], 'text':[]}
    for i in result['segments']:
        dict1['start'].append(int(i['start']))
        dict1['end'].append(int(i['end']))
        dict1['text'].append(i['text'])
    df = pd.DataFrame.from_dict(dict1)
    df.to_csv(f'{name}/subs.csv')
    vidcap = cv2.VideoCapture(f'''{name}/{input_file}''')
    success,image = vidcap.read()
    height = image.shape[0]
    width =image.shape[1]

    # Instantiate MoviePy subtitle generator with TextClip, subtitles, and SubtitlesClip
    generator = lambda txt: TextClip(
        txt,
        font='P052-Bold',
        fontsize=width / 30, # Adjust font size (increase for larger text)
        stroke_width=2, # Adjust stroke width
        color='yellow', # Change text color to yellow
        bg_color='transparent', # Set background color to transparent
        size=(width, height * 0.25),
        method='caption'
    )
    subs = tuple(zip(tuple(zip(df['start'].values, df['end'].values)), df['text'].values))
    subtitles = SubtitlesClip(subs, generator)

    # Ff the file was on youtube, add the captions to the downloaded video
    if download:
        video = VideoFileClip(f'{name}/{input_file}')
        final = CompositeVideoClip([video, subtitles.set_pos(('center','bottom'))])
        final.write_videofile(f'{output}', fps=video.fps, remove_temp=True, codec="libx264", audio_codec="aac")
    else:
        # If the file was a local upload:
        video = VideoFileClip(uploaded_vid)
        final = CompositeVideoClip([video, subtitles.set_pos(('center','bottom'))])
        final.write_videofile(f'{output}', fps=video.fps, remove_temp=True, codec="libx264", audio_codec="aac")

    return f'{output}'

## Declare relevant variables

In [ ]:
# Options for youtube download to ensure we get a high quality audio file extraction.
# This is key, as extracting from the video in the same download seemed to significantly affect caption Word Error Rate in our experiments.
# Only modify these if needed. Lowered audio quality may inhibit the transcription's word error rate.
opts_aud = {
    'format': 'mp3/bestaudio/best',
    'keep-video':True}

# Options for youtube video to get right video file for final output
opts_vid = {'format': 'mp4/bestvideo/best'}

# Youtube URL
URL = 'https://m.youtube.com/watch?v=I5gPddBRHjU&pp=ygUd2LPYrdixINmF2LfZhNio24wg2LHYp9iv24zZiCA%3D'# @param{type:'string'}
# The Hobbit Smaug in many languages

#URL = 'https://youtu.be/C-WFK7iKPb0?feature=shared' # steamed hams in many languages, sample link

# Generate subtitles

To autocaption our video, we just simply fill in the fields below with the relevant values.

The only required change is to the URL value if we would like that to be a different video from the sample.

> Note: If we run into an error, we can try restarting the kernel and running these 3 code cells again. It is unclear why this happens, but MoviePy seems to require a restart to the kernel occasionally.

In [ ]:
name = 'run2'
Video_patch = subtitle_video(
    download=True,
    #uploaded_vid='dune.mp4',     # path to local file
    url = URL,
    name = folder_path , #name,
    aud_opts = opts_aud,
    vid_opts = opts_vid,   # Video download settings
    model_type = 'large', # change to 'large' if you want more accurate results,
                           #change to 'medium.en' or 'large.en' for all english language tasks,
                           #and change to 'small' or 'base' for faster inference
    audio_file = "audio.mp3",
    input_file = 'dune.mp4',
    output = 'output.mp4',
    lang = 'en')

In [ ]:
from IPython.display import Video
Video(f'{Video_patch}',embed=True)